In [5]:
%matplotlib inline
import numpy as np
import cv2
import matplotlib as mpl
from matplotlib import pyplot as plt
import os

In [6]:
#Loading Variables
#orange_signs_list = os.listdir('./Img/RawOrangeSignImgs/')
#coco_imgs_list = os.listdir('./coco_dataset/images/chosenImages')
orange_signs_list = ['R-2_000.png']
coco_imgs_list = ['000000000025.jpg']



In [9]:
def applyErosion(mask):
    kernel = np.ones((5,5), np.uint8)
    mask = cv2.erode(mask, kernel, iterations=1)
    return mask

def createTemplate(orange_signs_list):
    templates_dict = {}
    for name in orange_signs_list:
        name = name[:-4:] # remove .png
        sign_type = name[:-4:] # select the sign type
        sign_number = name[-4::] # select the number of this sign type
        img = cv2.imread(f'./Img/RawOrangeSignImgs/{sign_type + sign_number}.png')
        #cv2.imshow("Img", img)
        #print('Img Shape:', img.shape)     

        hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV) 
        # print("HSV Img Shape: ", hsv_img.shape)

        lower_limit = np.array([12, 255, 255]) #Background HSV Color to create mask
        upper_limit = np.array([12, 255, 255]) #Background HSV Color to create mask

        mask = cv2.inRange(hsv_img, lower_limit, upper_limit)
        mask_inv = cv2.bitwise_not(mask)
        mask_inv = applyErosion(mask_inv) #Erode the mask to remove noise
        #print("Mask-Inv Shape:" + mask_inv.shape)  #should be (h, w, 3)

        contours, hierarchy = cv2.findContours(mask_inv, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        #print(np.asarray(contours).shape) # (1, 86, 1, 2) - 1 array, com 86 listas, cada lista com 1 sublistas com 2 elementos.
        valid_contours = []

        #Retira ruídos e monta o contorno principal
        for c in contours:
            area = cv2.contourArea(c)
            if area < 600:
                continue
            
            valid_contours.append(c)

        #Une a máscara invertida sobre a imagem principal para destacar a placa 
        sign = cv2.bitwise_and(img, img, mask= mask_inv)
        sign_rgba = cv2.cvtColor(sign, cv2.COLOR_BGR2BGRA)  #Convert to RGBA
        sign_rgba[:,:,3] = mask_inv
        # print(img_sign.shape) #(287, 124, 3)
        # cv2.imshow('Sign', sign_rgba)
        # cv2.waitKey(0)

        if not templates_dict.get(sign_type):
            templates_dict[sign_type] = [(sign_number, sign_rgba)]
        else:
            templates_dict[sign_type].append((sign_number, sign_rgba))

    return templates_dict
        
templates_dict = createTemplate(orange_signs_list)

In [10]:
def createSampleImages(templates_dict, coco_imgs_list):
    img_list = []
    for name in coco_imgs_list:
        name = name[:-4:] # remove .jpg
        coco_img = cv2.imread(f'./coco_dataset/images/chosenImages/{name}.jpg')
        if(coco_img.shape[0] > coco_img.shape[1]):    #Eliminate images with height greater than width (portraits)   
            continue
        #print(coco_img.shape)

        coco_img = cv2.resize(coco_img, (1366, 768), interpolation = cv2.INTER_LINEAR)
        #coco_img = cv2.cvtColor(coco_img, cv2.COLOR_BGR2HSV)
        
        x_offset=y_offset=50     #Change these to change the position of the sign

        for sign_type in templates_dict:
            for i, template_tuple in enumerate(templates_dict[sign_type]):
                sign_number = template_tuple[0] 
                template = template_tuple[1]
                #print(template.shape)
                
                #Using the offset to drag the image down and to the left    
                y1, y2 = y_offset, y_offset + template.shape[0]
                x1, x2 = x_offset, x_offset + template.shape[1]

                alpha_s = template[:, :, 3] / 255.0
                alpha_l = 1.0 - alpha_s
                for c in range(0, 3):
                    coco_img[y1:y2, x1:x2, c] = (alpha_s * template[:, :, c] +
                                            alpha_l * coco_img[y1:y2, x1:x2, c])
                                
        img_list.append(coco_img)
        

    cv2.imshow('Img', img_list[0])
    cv2.waitKey(0)

createSampleImages(templates_dict, coco_imgs_list)

In [ ]:
array = [array([[[  5,   4]], [[  6,   5]], [[  6,   6]],[[  8,   8]]], dtype=int32)]
